# Gen-LM experiments

In this notebook I mess around with gen-lm.

In [1]:
from genlm.control import PromptedLLM, BoolFSA, AWRS

INFO 09-23 17:36:11 [__init__.py:235] Automatically detected platform cuda.


In [ ]:
# Create a language model potential.
llm = PromptedLLM.from_name("gpt2")
llm.set_prompt_from_str("Here is my honest opinion:")

In [ ]:
# Create a finite-state automaton potential using a regular expression.
fsa = BoolFSA.from_regex(r" SMC is (🔥🔥|😍😍|🤌🤌) with LMs")

In [ ]:
# Coerce the FSA so that it operates on the token type of the language model.
coerced_fsa = fsa.coerce(llm, f=b"".join)

In [ ]:
# Create a token sampler that combines the language model and FSA.
token_sampler = AWRS(llm, coerced_fsa)

In [ ]:
# Check for CUDA-related package versions
import subprocess
import sys

def run_cmd(cmd):
    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        return result.stdout.strip()
    except:
        return "Command failed"

print("Python package versions:")
print("=" * 30)

packages = [
    "torch", "torchvision", "torchaudio", 
    "vllm", "triton", "xformers",
    "transformers", "accelerate"
]

for pkg in packages:
    try:
        version = __import__(pkg).__version__
        print(f"{pkg}: {version}")
    except ImportError:
        print(f"{pkg}: Not installed")
    except AttributeError:
        print(f"{pkg}: Version unknown")

# Check pip installations
print("\nCUDA-related pip packages:")
print("=" * 30)
print(run_cmd("pip list | grep -i cuda"))
print(run_cmd("pip list | grep -i torch"))

In [ ]:
# Generate text using SMC.
# Generation is asynchronous; use `await` if calling in an async context (like in an async
# function or in a Jupyter notebook) and `asyncio.run(token_sampler.smc(...))` otherwise.
sequences = await token_sampler.smc(
    n_particles=10, # Number of candidate sequences to maintain
    ess_threshold=0.5, # Threshold for resampling
    max_tokens=30, # Maximum sequence length
    verbosity=1 # Print particles at each step
)

In [ ]:
sequences.decoded_posterior
# Example output:
# {
#   ' SMC is 🔥🔥 with LMs': 1.0,
# }